# Best Models for Predicting California Housing Prices

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import  ElasticNet
from sklearn.linear_model import  Ridge
from sklearn.model_selection import cross_val_score
import  statsmodels.formula.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import  Lasso
from sklearn.model_selection import GridSearchCV
import matplotlib
from sklearn.decomposition import  PCA
from sklearn.cluster import  KMeans

%matplotlib inline


### Uploading 


In [2]:
# The code was removed by Watson Studio for sharing.

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### Data Cleaning Time

In [3]:
print('Shape of the dataframe is: \n', dh.shape)
print('Columns of the dataframe include: \n', dh.columns)

Shape of the dataframe is: 
 (20640, 10)
Columns of the dataframe include: 
 Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')


In [4]:
dh.dropna(axis = 0, inplace = True)    ### Clean data, drop the row with NaN value
dh = dh.rename(columns = {'longitude':'long', 'latitude':'lat', 'housing_median_age':'age','total_rooms':'rooms','total_bedrooms':'bedrooms','population':'pop','median_income':'income','median_house_value':'price','ocean_proximity':'water'})

In [5]:
dhn = dh.sample(n = 10000)             #### Only randomly get 10000 samples from the 330000+ data, for quick reviewing
dhn.head()

,long,lat,age,rooms,bedrooms,pop,households,income,price,water
8753,-118.36,33.82,26.0,5166.0,1313.0,2738.0,1239.0,3.3565,360800.0,<1H OCEAN
10914,-117.87,33.74,31.0,2338.0,652.0,3289.0,631.0,2.6734,158500.0,<1H OCEAN
6707,-118.15,34.14,27.0,1499.0,426.0,755.0,414.0,3.8750,258300.0,<1H OCEAN
1007,-121.75,37.68,35.0,1755.0,299.0,702.0,263.0,5.2443,183400.0,INLAND
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0,NEAR OCEAN


In [6]:
dhn = dhn.drop(columns = ['water'])
dhn.head()

,long,lat,age,rooms,bedrooms,pop,households,income,price
8753,-118.36,33.82,26.0,5166.0,1313.0,2738.0,1239.0,3.3565,360800.0
10914,-117.87,33.74,31.0,2338.0,652.0,3289.0,631.0,2.6734,158500.0
6707,-118.15,34.14,27.0,1499.0,426.0,755.0,414.0,3.8750,258300.0
1007,-121.75,37.68,35.0,1755.0,299.0,702.0,263.0,5.2443,183400.0
14053,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,112500.0


In [7]:
dh.dtypes  ## Check type of each columns, change object to int for future analysis

long          float64
lat           float64
age           float64
rooms         float64
bedrooms      float64
pop           float64
households    float64
income        float64
price         float64
water          object
dtype: object

In [8]:
dh.describe()

,long,lat,age,rooms,bedrooms,pop,households,income,price
count,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000,20433.000000
mean,-119.570689,35.633221,28.633094,2636.504233,537.870553,1424.946949,499.433465,3.871162,206864.413155
std,2.003578,2.136348,12.591805,2185.269567,421.385070,1133.208490,382.299226,1.899291,115435.667099
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1450.000000,296.000000,787.000000,280.000000,2.563700,119500.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.536500,179700.000000
75%,-118.010000,37.720000,37.000000,3143.000000,647.000000,1722.000000,604.000000,4.744000,264700.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [9]:
dhn[['long', 'lat', 'age','rooms','bedrooms','pop','households','income','price']] = dhn[['long', 'lat', 'age','rooms','bedrooms','pop','households','income','price']].astype(int)

In [10]:
#### data normalization for totalPrice, square
maxPrice = max(dhn['price'])
dhn['price'] = dhn['price'] / maxPrice

### Visualize some basic trends

In [ ]:
fig, axs = plt.subplots(ncols=4, figsize = (30,5))
sns.boxplot(x='age', y='price', data=dhn, ax=axs[0])
sns.boxplot(x='rooms', y='price', data=dhn, ax=axs[1])
sns.boxplot(x='bedrooms', y='price', data=dhn, ax=axs[2])
sns.boxplot(x='income', y='price', data=dhn, ax=axs[3])

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(dh[dh['bedrooms'].notnull()]['bedrooms'],bins=10,color='blue')#histogram of bedrooms
#remove outliars
(dh['bedrooms']>4000).sum()
plt.title('Bedroom Frequency')
plt.xlabel('Total bedrooms')
plt.ylabel('Frequency')

In [ ]:
def calcCategoricalMedian(x):
    """ fill the missing values of bedrooms based on categories of water"""
    uniqueWater=x['water'].unique()
    for i in uniqueWater:
        median=x[x['water']==i]['bedrooms'].median()
        x.loc[x['water']==i,'bedrooms'] =  x[x['water']==i]['bedrooms'].fillna(median)
calcCategoricalMedian(dh)

In [ ]:
dh.plot(kind="scatter",x="long",y="lat",alpha=0.4,
             s=dh["pop"]/100,label="population",figsize=(10,7),
             c="price",cmap=plt.get_cmap("jet"),colorbar=True, 
            )
plt.legend()

In [ ]:
sns.distplot(dh.price)

In [ ]:
plt.figure(figsize=(10,7))

plt.scatter(dh['pop'],dh['price'],c=dh['price'],s=dh['income']*50)
plt.colorbar
plt.title('Area Population v House Value' )
plt.xlabel('population')
plt.ylabel('house value')
plt.plot()

In [ ]:
plt.figure(figsize=(10,7))
sns.heatmap(cbar=False,annot=True,data=dh.corr())
plt.title('% Corelation Matrix')
plt.show()

In [ ]:
corrMatrix=dh.corr()
corrMatrix["price"].sort_values(ascending=False) 

In [ ]:
plt.figure(figsize=(10,7))
sns.boxplot(data=dh,x='water',y='price')
plt.plot()

### Data Selection

In [ ]:
dh=pd.concat([pd.get_dummies(dh['water'],drop_first=True),dh],axis=1).drop('water',axis=1)
dh['Income per Population']=dh['income']/(dh['pop']-dh['households'])
dh['Bedrooms per House']=dh['bedrooms']/dh['rooms']
dh['Houses per Population']=dh['households']/dh['pop']

In [ ]:
def buildingType(x):
    if x<=10:
        return "new"
    elif x<=30:
        return 'mid age'
    else:
        return 'old'
df=pd.concat([dh,pd.get_dummies(dh['age'].apply(buildingType),drop_first=True)],axis=1)

In [ ]:
x=dh.drop('price',axis=1).values
y=dh['price'].values

### More Data Cleaning/ Pre Processing

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

print("number of training samples:",x_train.shape[0])
print("number of test samples:", x_test.shape[0])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
ms=MinMaxScaler()
x_train=ms.fit_transform(x_train)
x_test=ms.transform(x_test)

In [ ]:
def variance(x):
    total=0
    hlist=[]
    for i in np.arange(0,x_train.shape[1]):
        p=PCA(n_components=i+1)
        p.fit(x)
        total=total+p.explained_variance_ratio_[i]
        hlist.append(total)
        
    return hlist
xtrainvariance=list(map(lambda x:x*100,variance(x_train)))

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(np.arange(1,x_train.shape[1]+1),xtrainvariance,marker='x',markerfacecolor='red',lw=6)
plt.xlabel('number of components')
plt.ylabel('comulative variance %')
plt.title('comulative variance ratio of p.c.a components')

In [ ]:
pca=PCA(n_components=2)
pca.fit(x_train)
ptrain=pca.fit_transform(x_train)

In [ ]:
lm = LinearRegression()
lm.fit(X,Y)
print(lm.coef_)

In [ ]:
bestk=[]
for i in range(1,10):
    k=KMeans(n_clusters=i)
    k.fit(x_train)
    bestk.append(k.inertia_)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(np.arange(1,len(bestk)+1),bestk,marker='o',markerfacecolor='blue',lw=5,color='red')
plt.title('Elbow curve')
plt.xlabel('number of clusters')
plt.ylabel('WWSS')
plt.show()

In [ ]:
k=KMeans(n_clusters=4)
kpred=k.fit_predict(x_train)

In [ ]:
plt.figure(figsize=(9,6))
color=['purple','blue','green','pink']
for i in range(3):
    plt.scatter(ptrain[kpred==i][:,0],ptrain[kpred==i][:,1],c=color[i])
    plt.scatter(k.cluster_centers_[i,0],k.cluster_centers_[i,1],marker='x')
    plt.xlabel('pc1')
    plt.ylabel('pc2')

In [ ]:
matplotlib.rcParams.update({'font.size': 12})
pca=PCA(n_components=None)
pca.fit(x_train)
plt.figure(figsize=(20,15))
sns.heatmap(pca.components_,annot=True,xticklabels=dh.drop('price',axis=1).columns,yticklabels=[str(i) for i in range(1,len(dh.columns))])
plt.xlabel('Features')
plt.ylabel('Components')
plt.title('Relation Matrix')
plt.show()
matplotlib.rcParams.update({'font.size': 10})

### Simple Linear Regression Model

In [ ]:
### predict by using x_test
yhat_lm = lm.predict(x_test)
ax1 = sns.distplot(y_test, hist= False, color='b', label = "Actual Value")
sns.distplot(yhat_lm, hist=False, color = "r", label ="Predict price by Simple Linear Regression", ax=ax1)

In [ ]:
print('MSE for SLR is: ', mean_squared_error(y_test, yhat_lm))
print('R score for SLR is: ', lm.score(x_test, y_test))

### A PolyRegression Model

In [ ]:
Rsqu_test = [] ## create empty list
order =[1,2,3,4]    ## contains different polynomial orders
for n in order:
	pr = PolynomialFeatures(degree = n)
	x_train_pr =pr.fit_transform(x_train)
	x_test_pr = pr.fit_transform(x_test)
	lm.fit(x_train_pr, y_train)
	Rsqu_test.append(lm.score(x_test_pr, y_test))
print('R square score for different polynomial orders are: ',Rsqu_test)

In [ ]:
def regresssor_model(x,y,estimator):
   
    regressor=estimator()
    regressor.fit(x,y)
    lr_rmse=np.sqrt(mean_squared_error(y,regressor.predict(x)))
    cv_regressor=cross_val_score(cv=10,X=x,y=y,estimator=regressor,scoring='r2')
    print('The cross validated accuracy  - '+str(100*cv_regressor.mean()))
    print('The corss validated variance is - '+str(100*cv_regressor.std()))
    return regressor

In [ ]:
def topoly(degree,x_train,x_test):
    poly=PolynomialFeatures(degree=degree)
    X=poly.fit_transform(x_train)
    x=poly.fit_transform(x_test)
    return (X,x)

In [ ]:
def evaluate(ypred,ytest,regressor):
    plt.figure(figsize=(15,8))
    plt.xlabel('(ytest) - (ypred)')
    plt.ylabel('frequency')
    plt.title('residual plot')
    plt.hist(ytest-ypred)
    print("root mean squared error for test data   is "+str(np.sqrt(mean_squared_error(ytest,ypred))))
    plt.show()

In [ ]:
xtrainpoly,xtestpoly=topoly(3,x_train,x_test)
l=regresssor_model(xtrainpoly,y_train,LinearRegression)
evaluate(l.predict(xtestpoly),y_test,l)

In [ ]:
xtrainpoly,xtestpoly=topoly(2,x_train[:,11:12],x_test[:,11:12])
l=regresssor_model(xtrainpoly,y_train,LinearRegression)
evaluate(l.predict(xtestpoly),y_test,l)

### A ridge model

In [ ]:
check =regresssor_model(x_train,y_train,Ridge)
evaluate(check.predict(x_test),y_test,check)
plt.figure(figsize=(10,7))
plt.bar(np.arange(len(check.coef_)),check.coef_,color='blue')
plt.xlabel('coefficients')
plt.ylabel('coefficients value')
plt.title('coeff graph')

In [ ]:
RidgeModel = Ridge(alpha = 0.01)
RidgeModel.fit(x_train,y_train)
yhat_ridge = RidgeModel.predict(x_test)


print('MSE for RidgeModel is: ', mean_squared_error(y_test, yhat_ridge))
print('R score for RidgeModel is: ', RidgeModel.score(x_test, y_test))

### predict by using x_test
ax2 = sns.distplot(y_test, hist= False, color='r', label = "Actual Value")
sns.distplot(yhat_ridge, hist=False, color = "b", label ="Predict price by RidgeModel", ax=ax2)

### For polynomial ridgeModel

In [ ]:
pr = PolynomialFeatures(degree=2)
x_train_pr = pr.fit_transform(x_train)
x_test_pr = pr.fit_transform(x_test)
RidgeModel_poly = Ridge(alpha = 0.01)
RidgeModel_poly.fit(x_train_pr, y_train)
yhat_ridge_poly = RidgeModel_poly.predict(x_test_pr)

print('MSE for RidgeModel-poly is: ', mean_squared_error(y_test, yhat_ridge_poly))
print('R score for RidgeModel-poly is: ', RidgeModel_poly.score(x_test_pr, y_test))

### predict by using x_test
ax3 = sns.distplot(y_test, hist= False, color='r', label = "Actual Value")
sns.distplot(yhat_ridge_poly, hist=False, color = "b", label ="Predict price by RidgeModel_poly", ax=ax3)

In [ ]:
dffs = dh.sample(n = 500) 

### Decision Trees

In [ ]:
dtree=regresssor_model(x_train,y_train,DecisionTreeRegressor)
dtree.fit(x_train,y_train)
print('mean squared errror is',end='\t-')
np.sqrt(mean_squared_error(y_test,dtree.predict(x_test)))

In [ ]:
par=[{
            
            'max_depth':[2,3,4,5,6,10,20,30,40,50,60,70,100],
            'min_samples_split':[2,3,4,7,10,12],
            'min_samples_leaf' :[1,3,5,10,15,20,25],
            'max_features':['sqrt','log2'],
            
        }
        ]

from sklearn.model_selection import GridSearchCV
gc=GridSearchCV(dtree,par,cv=10,scoring='r2',n_jobs=-1)
gc.fit(x_train,y_train)
gc.best_estimator_

In [ ]:
gc.best_score_
dt=gc.best_estimator_
dt.fit(x_train,y_train)
np.sqrt(mean_squared_error(y_test,dtree.predict(x_test)))

In [ ]:
plt.figure(figsize=(10,8))
data=pd.DataFrame({'feature':dh.columns[dh.columns!='price'].values,"importance":dtree.feature_importances_})
sns.barplot(data=data,y='feature',x='importance')
plt.title('feature importance')

### Random Forest

In [ ]:
rg=RandomForestRegressor(n_estimators=50)
rg.fit(x_train,y_train)

In [ ]:
print(np.sqrt(mean_squared_error(y_test,rg.predict(x_test))))
print(rg.score(x_test,y_test))

In [ ]:
plt.figure(figsize=(10,7))
plt.hist(y_test-rg.predict(x_test))

In [ ]:
params=[{
            'n_estimators':[20,30,70,50,100,200,300,400,600,650,630,680],
            'max_depth':[10,20,30,40,50,60,70,100],
            'min_samples_split':[2,3,4,5,10],
            'min_samples_leaf' :[1,2,5,7,10],
            'bootstrap':[True,False],
            'max_features':['sqrt','auto']
            
            
        }
]

In [ ]:
rg=RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=70,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=630, n_jobs=1,
           oob_score=False, verbose=0, warm_start=False)
rg.fit(x_train,y_train)
np.sqrt(mean_squared_error(y_test,rg.predict(x_test)))

In [ ]:
np.sqrt(mean_squared_error(y_test,rg.predict(x_test)))

In [ ]:
plt.figure(figsize=(12,7))
plt.title('Residual Plot')
plt.hist(y_test-rg.predict(x_test))
plt.show()

In [ ]:
rg.score(x_test,y_test)

In [ ]:
plt.figure(figsize=(9,6))
plt.title('Feature Importance')

sns.barplot(data={'importance':rg.feature_importances_,'feature':dh.columns[dh.columns!='price']},y='feature',x='importance')

In [ ]:
rg=RandomForestRegressor(n_estimators=400)
rg.fit(x_train[:,11:12],y_train)

In [ ]:
xt=np.arange(min(x_test[:,11]),max(x_test[:,11]),0.005)
xt=xt.reshape(len(xt),1)

## Discussion

Random forest would be the best model for this data set because of the low mean squared error and high r squared it produces.
With the most important features being average income of area, and whether or not a house is located inland.